# [Lab3] SageMaker Deployment

In [ ]:
%store -r

In [ ]:
import sagemaker
import boto3
from time import gmtime, strftime

boto_session = boto3.Session()
sess = sagemaker.Session()

## Local Test

### Loading Test Dataset

In [ ]:
!aws s3 cp $test_path/test_x.csv tmp/test_x.csv
!aws s3 cp $test_path/test_y.csv tmp/test_y.csv

In [ ]:
import pandas as pd
import xgboost as xgb
import numpy as np

test_x = pd.read_csv('tmp/test_x.csv', names=[f'{i}' for i in range(59)])
test_y = pd.read_csv('tmp/test_y.csv', names=['y'])

### Test - Model1

In [ ]:
import mlflow

model_uri = registered_model_version_1.source
model = mlflow.xgboost.load_model(model_uri)

dtest = xgb.DMatrix(test_x)
predictions = np.array(model.predict(dtest), dtype=float).squeeze()
predictions

In [ ]:
pd.crosstab(index=test_y['y'].values, columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])

### Test - Model2

In [ ]:
model_uri = registered_model_version_2.source
model = mlflow.xgboost.load_model(model_uri)

dtest = xgb.DMatrix(test_x)
predictions = np.array(model.predict(dtest), dtype=float).squeeze()
predictions

In [ ]:
pd.crosstab(index=test_y['y'].values, columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])

In [ ]:
mlflow.set_tracking_uri(mlflow_arn)

run = mlflow.get_run(run_id_1)

In [ ]:
from sklearn import metrics
from sklearn.metrics import RocCurveDisplay
import matplotlib.pyplot as plt

def plot_confusion_matrix(
    cm, class_names, title="Confusion matrix", cmap=plt.cm.Blues, normalize=False
):
    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation="nearest", cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    ax.set(
        xticks=np.arange(cm.shape[1]),
        yticks=np.arange(cm.shape[0]),
        ylim=(cm.shape[0] - 0.5, -0.5),
        xticklabels=class_names,
        yticklabels=class_names,
        title=title,
        ylabel="Ground truth label",
        xlabel="Predicted label",
    )

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=30, ha="right", rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = ".2f"
    thresh = cm.max() / 2.0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(
                j,
                i,
                format(cm[i, j], fmt),
                ha="center",
                va="center",
                color="white" if cm[i, j] > thresh else "black",
            )
    fig.tight_layout()
    return ax, fig

In [ ]:
class_names = ["no", "yes"]
confusion_matrix = metrics.confusion_matrix(test_y, np.round(predictions))
ax, fig = plot_confusion_matrix(confusion_matrix, class_names)

In [ ]:
print(f"Log confusion matrix to the model {registered_model_version_1.name} version {registered_model_version_1.version}")
mlflow.set_experiment(experiment_name)
with mlflow.start_run(run_id=registered_model_version_1.run_id):
    mlflow.log_figure(fig, "confusion_matrix.png")

## Deploy Model with a SageMaker Real-time Inference endpoint

In [ ]:
from time import gmtime, strftime

In [ ]:
container = sagemaker.image_uris.retrieve(region=boto3.Session().region_name, framework='xgboost', version='1.7-1')

In [ ]:
from sagemaker.model import Model

run = mlflow.get_run(run_id_1)
model_data = run.data.params["model_origin"]
xgb1_model = Model(model_data=model_data,
                   image_uri=container,  
                   role=role,  
                   sagemaker_session=sess,
                   name="xgb1-model")

run = mlflow.get_run(run_id_2)
model_data = run.data.params["model_origin"]
xgb2_model = Model(model_data=model_data,
                   image_uri=container,  
                   role=role,  
                   sagemaker_session=sess,
                   name="xgb2-model")

xgb1_model.create()
xgb2_model.create()

In [ ]:
endpoint_config_name = f"my-endpoint-config-{strftime('%d-%H-%M-%S', gmtime())}"

response = sess.sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "Variant1",
            "ModelName": "xgb1-model",
            "InstanceType": "ml.m5.large",
            "InitialInstanceCount": 1,
            "InitialVariantWeight": 1,  # 50%
            "RoutingConfig": {
                "RoutingStrategy": "LEAST_OUTSTANDING_REQUESTS"
            }
        },
        {
            "VariantName": "Variant2",
            "ModelName": "xgb2-model",
            "InstanceType": "ml.m5.large",
            "InitialInstanceCount": 1,
            "InitialVariantWeight": 1,  # 50%
            "RoutingConfig": {
                "RoutingStrategy": "LEAST_OUTSTANDING_REQUESTS"
            }
        }
    ]
)


In [ ]:
endpoint_name = f"my-endpoint-{strftime('%d-%H-%M-%S', gmtime())}"

response = sess.sagemaker_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)

In [ ]:
%store endpoint_name

### Please wait while the endpoint is being deployed.

In [ ]:
waiter = sess.sagemaker_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

## Test Endpoint

In [ ]:
import json 

test_x = pd.read_csv('tmp/test_x.csv', names=[f'{i}' for i in range(59)], nrows=1)
record = ','.join(map(str, test_x.iloc[0]))

sm_runtime = boto3.client('runtime.sagemaker')

for i in range(10):
    response = sm_runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='text/csv',  
        Body=record 
    )
    result = response['Body'].read().decode('utf-8')

    try:
        parsed_result = json.loads(result)
        print('Parsed result:', parsed_result)
    except json.JSONDecodeError:
        print('Result is not valid JSON')

In [ ]:
update_endpoint_config_response = sess.sagemaker_client.update_endpoint_weights_and_capacities(
    EndpointName=endpoint_name,
    DesiredWeightsAndCapacities=[
        {
            "VariantName": "Variant1",
            "DesiredWeight": 0,
        },
        {
            "VariantName": "Variant2",
            "DesiredWeight": 100,
        }
    ]
)

In [ ]:
test_x = pd.read_csv('tmp/test_x.csv', names=[f'{i}' for i in range(59)], nrows=1)
record = ','.join(map(str, test_x.iloc[0]))

sm_runtime = boto3.client('runtime.sagemaker')

for i in range(10):
    response = sm_runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='text/csv',  
        Body=record 
    )
    result = response['Body'].read().decode('utf-8')

    try:
        parsed_result = json.loads(result)
        print('Parsed result:', parsed_result)
    except json.JSONDecodeError:
        print('Result is not valid JSON')

## Remember to delete the endpoint when it's no longer needed

In [ ]:
# sm_client.delete_endpoint(
#     EndpointName=endpoint_name
# )